# ポケモン基本計算まとめ

- ステータス実数値計算


## 実行環境確認

Pythonの仮想環境確認

In [10]:
!which python

/home/shouhei/anaconda3/envs/6.86x/bin/python


In [3]:
!which pip

/home/shouhei/anaconda3/envs/6.86x/bin/pip


## パッケージアップデート

- numpy
- matplotlib
- pandas

In [4]:
!pip list | grep numpy

numpy              1.16.4   


In [8]:
!pip install --upgrade numpy

     |████████████████████████████████| 20.0MB 2.6MB/s 
  Found existing installation: numpy 1.16.4
    Uninstalling numpy-1.16.4:
      Successfully uninstalled numpy-1.16.4


In [15]:
!pip list | grep numpy

numpy              1.17.4   


In [14]:
!pip install --upgrade matplotlib

     |████████████████████████████████| 13.1MB 2.2MB/s 
  Found existing installation: matplotlib 3.1.0
    Uninstalling matplotlib-3.1.0:
      Successfully uninstalled matplotlib-3.1.0


In [16]:
!pip list | grep matplotlib

matplotlib         3.1.2    


---

## ポケモン実数値計算

H, A, B, C, D, Eの実数値計算

### 基本計算式

計算式は以下の通り。

- $x_i$ を `種族値`
    - $i = 0, 1, 2, 3, 4, 5$ 
        - $x_0 = H$ : [1..255]
        - $x_1 = A$ : [10..170]
        - $x_2 = B$ : [15..250]
        - $x_3 = C$ : [35..145]
        - $x_4 = D$ : [95..250]
        - $x_5 = S$ : [5..160]
- $y$ を `個体値` : [0..31]
- $z$ を `努力値` : [0..252]
- $\rm Lv$ を `レベル` : [1..100]

Hの場合

$ v_0 = \{(x_0\cdot 2 + y + \frac{z}{4}) \times (\frac{ \rm Lv }{100})\} + (10+ \rm Lv)$ 

A, B, C, D, Eの場合

$ v_i = [\{(x_i \cdot 2 + y + \frac{z}{4})  \times (\frac{ \rm Lv }{100})\} + 5] $ 

補正前実数値ベクトルを$\mathbf{v}$、 性格補正ベクトルを$\mathbf{u}$とする

$ \mathbf{v} = 
\begin {pmatrix} v_0 \\
                 v_1 \\
                 v_2 \\
                 v_3 \\
                 v_4 \\
                 v_5
\end {pmatrix} \in \mathbb{I}^6 $

$ \mathbf{u} = 
\begin {pmatrix} 1.0 \\
                 u_1 \\
                 u_2 \\
                 u_3 \\
                 u_4 \\
                 u_5
\end {pmatrix}  \in \mathbb{R}^6 $

よって、補正後実数値はelement-wizeの積で表現できる

$ \mathbf{v} \odot \mathbf{u}  \in \mathbb{I}^6 $

また、実数値合計は内積で表現できる

$ \mathbf{v}^T \cdot \mathbf{u} \in \mathbb{I}$

#### 種族値入力

In [35]:
import numpy as np

H = int(input("H: "))
A = int(input("A: "))
B = int(input("B: "))
C = int(input("C: "))
D = int(input("D: "))
S = int(input("S: "))

H:  70
A:  110
B:  70
C:  115
D:  70
S:  90


In [125]:
def calcActualH(x, y, z, level):
    return int((((x*2 + y + z/4) * (level/100)) + (10 + level)))

def calcActualABCDS(x, y, z, level):
    return int(((x*2 + y + z/4) * (level/100)) + 5)

def createVectorV(H, A, B, C, D, S, 
                  y_H,y_A,y_B,y_C,y_D,y_S, 
                  z_H,z_A,z_B,z_C,z_D,z_S,
                  level):
    return np.array(
        [calcActualH(H,y_H,z_H,level),
         calcActualABCDS(A,y_A,z_A,level),
         calcActualABCDS(B,y_B,z_B,level),
         calcActualABCDS(C,y_C,z_C,level),
         calcActualABCDS(D,y_D,z_D,level),
         calcActualABCDS(S,y_S,z_S,level)
        ]
    )

#### 性格補正ベクトル $\mathbf{u}$

In [114]:
# 物理アタッカー いじっぱり
u_0 = np.array([1.0, 1.1, 1.0, 0.9, 1.0, 1.0])
# 高速物理アタッカー  ようき
u_1 = np.array([1.0, 1.0, 1.0, 0.9, 1.0, 1.1])
# 特殊アタッカー ひかえめ
u_2 = np.array([1.0, 0.9, 1.0, 1.1, 1.0, 1.0])
# 高速特殊アタッカー おくびょう
u_3 = np.array([1.0, 0.9, 1.0, 1.0, 1.0, 1.1])
# 物理受け ずぶとい
u_4 = np.array([1.0, 0.9, 1.1, 1.0, 1.0, 1.0])
# 物理受け わんぱく
u_5 = np.array([1.0, 1.0, 1.1, 0.9, 1.0, 1.0])
# 特殊受け おだやか
u_6 = np.array([1.0, 0.9, 1.0, 1.0, 1.1, 1.0])
# 特殊受け しんちょう
u_7 = np.array([1.0, 1.0, 1.0, 0.9, 1.1, 1.0])

In [128]:
import pandas as pd

header = np.array(["H","A","B","C","D","S"])
index = np.array([
    "物理アタッカー",
    "高速物理アタッカー",
    "特殊アタッカー",
    "高速特殊アタッカー",
    "物理受け",
    "物理受け",
    "特殊受け",
    "特殊受け"
])

https://stackoverflow.com/questions/20763012/creating-a-pandas-dataframe-from-a-numpy-array-how-do-i-specify-the-index-colum

---

#### Case1. 種族値のみ

- 個体値逆6V : $y = 0$
- 努力値無振り : $z = 0$
- $\rm Lv = 50$

Hの場合

$x_0 + 60$ 

A, B, C, D, Eの場合

$(x_i + 5)$

In [131]:
v = createVectorV(H, A, B, C, D, S,
                  0, 0, 0, 0, 0, 0,
                  0, 0, 0, 0, 0, 0,
                  50)

v_0 = (v * u_0).astype(int)
v_1 = (v * u_1).astype(int)
v_2 = (v * u_2).astype(int)
v_3 = (v * u_3).astype(int)
v_4 = (v * u_4).astype(int)
v_5 = (v * u_5).astype(int)
v_6 = (v * u_6).astype(int)
v_7 = (v * u_7).astype(int)

data = np.array([header,v_0,v_1,v_2,v_3,v_4,v_5,v_6,v_7])

pd.DataFrame(data=data[1:,0:],    # [row:,col:]
             index=index,
             columns=data[0,0:])

,H,A,B,C,D,S
物理アタッカー,130,126,75,108,75,95
高速物理アタッカー,130,115,75,108,75,104
特殊アタッカー,130,103,75,132,75,95
高速特殊アタッカー,130,103,75,120,75,104
物理受け,130,103,82,120,75,95
物理受け,130,115,82,108,75,95
特殊受け,130,103,75,120,82,95
特殊受け,130,115,75,108,82,95


In [140]:
pd.DataFrame(np.array([v_0,v_1,v_2,v_3,v_4,v_5,v_6,v_7])).describe()

,0,1,2,3,4,5
count,8.0,8.000000,8.00000,8.000000,8.00000,8.00000
mean,130.0,110.375000,76.75000,115.500000,76.75000,97.25000
std,0.0,8.667468,3.24037,8.928286,3.24037,4.16619
min,130.0,103.000000,75.00000,108.000000,75.00000,95.00000
25%,130.0,103.000000,75.00000,108.000000,75.00000,95.00000
50%,130.0,109.000000,75.00000,114.000000,75.00000,95.00000
75%,130.0,115.000000,76.75000,120.000000,76.75000,97.25000
max,130.0,126.000000,82.00000,132.000000,82.00000,104.00000


---

#### Case2. 種族値 + 6V

Hの場合

$ (x + \frac{y}{2}) + 60$ 

A, B, C, D, Eの場合

$\{(x + \frac{y}{2}) + 5\} \times \alpha$

In [142]:
v = createVectorV(H, A, B, C, D, S,
                  31, 31, 31, 31, 31, 31,
                  0, 0, 0, 0, 0, 0,
                  50)

v_0 = (v * u_0).astype(int)
v_1 = (v * u_1).astype(int)
v_2 = (v * u_2).astype(int)
v_3 = (v * u_3).astype(int)
v_4 = (v * u_4).astype(int)
v_5 = (v * u_5).astype(int)
v_6 = (v * u_6).astype(int)
v_7 = (v * u_7).astype(int)

data = np.array([header,v_0,v_1,v_2,v_3,v_4,v_5,v_6,v_7])

pd.DataFrame(data=data[1:,0:],    # [row:,col:]
             index=index,
             columns=data[0,0:])

,H,A,B,C,D,S
物理アタッカー,145,143,90,121,90,110
高速物理アタッカー,145,130,90,121,90,121
特殊アタッカー,145,117,90,148,90,110
高速特殊アタッカー,145,117,90,135,90,121
物理受け,145,117,99,135,90,110
物理受け,145,130,99,121,90,110
特殊受け,145,117,90,135,99,110
特殊受け,145,130,90,121,99,110


In [143]:
pd.DataFrame(np.array([v_0,v_1,v_2,v_3,v_4,v_5,v_6,v_7])).describe()

,0,1,2,3,4,5
count,8.0,8.00000,8.00000,8.000000,8.00000,8.000000
mean,145.0,125.12500,92.25000,129.625000,92.25000,112.750000
std,0.0,9.67231,4.16619,10.155048,4.16619,5.092011
min,145.0,117.00000,90.00000,121.000000,90.00000,110.000000
25%,145.0,117.00000,90.00000,121.000000,90.00000,110.000000
50%,145.0,123.50000,90.00000,128.000000,90.00000,110.000000
75%,145.0,130.00000,92.25000,135.000000,92.25000,112.750000
max,145.0,143.00000,99.00000,148.000000,99.00000,121.000000


---

#### Case3. 種族値 + 6V + 基礎値全振り

- 個体値: 6V
- 基礎値:
    - A: 252
    - S: 252
    - H: 4
- Lv: 50

In [144]:
v = createVectorV(H, A, B, C, D, S,
                  31, 31, 31, 31, 31, 31,
                  4, 252, 0, 0, 0, 252,
                  50)

v_0 = (v * u_0).astype(int)
v_1 = (v * u_1).astype(int)
v_2 = (v * u_2).astype(int)
v_3 = (v * u_3).astype(int)
v_4 = (v * u_4).astype(int)
v_5 = (v * u_5).astype(int)
v_6 = (v * u_6).astype(int)
v_7 = (v * u_7).astype(int)

data = np.array([header,v_0,v_1,v_2,v_3,v_4,v_5,v_6,v_7])

pd.DataFrame(data=data[1:,0:],    # [row:,col:]
             index=index,
             columns=data[0,0:])

,H,A,B,C,D,S
物理アタッカー,146,178,90,121,90,142
高速物理アタッカー,146,162,90,121,90,156
特殊アタッカー,146,145,90,148,90,142
高速特殊アタッカー,146,145,90,135,90,156
物理受け,146,145,99,135,90,142
物理受け,146,162,99,121,90,142
特殊受け,146,145,90,135,99,142
特殊受け,146,162,90,121,99,142


In [145]:
pd.DataFrame(np.array([v_0,v_1,v_2,v_3,v_4,v_5,v_6,v_7])).describe()

,0,1,2,3,4,5
count,8.0,8.000000,8.00000,8.000000,8.00000,8.000000
mean,146.0,155.500000,92.25000,129.625000,92.25000,145.500000
std,0.0,12.386629,4.16619,10.155048,4.16619,6.480741
min,146.0,145.000000,90.00000,121.000000,90.00000,142.000000
25%,146.0,145.000000,90.00000,121.000000,90.00000,142.000000
50%,146.0,153.500000,90.00000,128.000000,90.00000,142.000000
75%,146.0,162.000000,92.25000,135.000000,92.25000,145.500000
max,146.0,178.000000,99.00000,148.000000,99.00000,156.000000


- 個体値: 6V
- 基礎値:
    - A: 248
    - S: 248
    - H: 4
- Lv: 50

In [146]:
v = createVectorV(H, A, B, C, D, S,
                  31, 31, 31, 31, 31, 31,
                  4, 248, 0, 0, 0, 248,
                  50)

v_0 = (v * u_0).astype(int)
v_1 = (v * u_1).astype(int)
v_2 = (v * u_2).astype(int)
v_3 = (v * u_3).astype(int)
v_4 = (v * u_4).astype(int)
v_5 = (v * u_5).astype(int)
v_6 = (v * u_6).astype(int)
v_7 = (v * u_7).astype(int)

data = np.array([header,v_0,v_1,v_2,v_3,v_4,v_5,v_6,v_7])

pd.DataFrame(data=data[1:,0:],    # [row:,col:]
             index=index,
             columns=data[0,0:])

,H,A,B,C,D,S
物理アタッカー,146,177,90,121,90,141
高速物理アタッカー,146,161,90,121,90,155
特殊アタッカー,146,144,90,148,90,141
高速特殊アタッカー,146,144,90,135,90,155
物理受け,146,144,99,135,90,141
物理受け,146,161,99,121,90,141
特殊受け,146,144,90,135,99,141
特殊受け,146,161,90,121,99,141


In [147]:
pd.DataFrame(np.array([v_0,v_1,v_2,v_3,v_4,v_5,v_6,v_7])).describe()

,0,1,2,3,4,5
count,8.0,8.000000,8.00000,8.000000,8.00000,8.000000
mean,146.0,154.500000,92.25000,129.625000,92.25000,144.500000
std,0.0,12.386629,4.16619,10.155048,4.16619,6.480741
min,146.0,144.000000,90.00000,121.000000,90.00000,141.000000
25%,146.0,144.000000,90.00000,121.000000,90.00000,141.000000
50%,146.0,152.500000,90.00000,128.000000,90.00000,141.000000
75%,146.0,161.000000,92.25000,135.000000,92.25000,144.500000
max,146.0,177.000000,99.00000,148.000000,99.00000,155.000000


#### Case4. 種族値 + 6V + 基礎値全振り + 性格補正 + もちもの